In [1]:
import chancelator
import re

In [2]:
new_ideas = chancelator.collect_ideas(first_time=False)

False
Search urls ['https://www.halfbakery.com/view/ftm=r259200:s=Qr:d=irq:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions']
Number of unique links to follow: 25
Processing...
 25 pages to collect
cc79faa9ec8cb5483aba6477a0d389d3
fe9604ccfa7e4593fae7795e03614d9b
5448d10619a0fb990d40ccba19bc3e42
f27b294b9eb80494aeeebbb0472f7fc3
fabbb9a8171fe0a30328d7d2687536ae
740557e39c64d4a0d3e8108a180a66af
53b87b2666b37d514f78b72054eb7d36
1ed4bca2f7d115711b0c396a8d21f24b
c571615148f8a869e9ac63701f5671df
662a4896a99246078063ef575b4b8c88
Read 10 of 25...
a55ff3862bea3fe12de048b6e57365d2
36867f712b282a0ab21c6865eaa3b009
e9afbce331dcde6084d64cc23b638962
328b60abdd2bb1b6b274ea7dd9bd2dee
9854808fd506f11da20d1a327bd55235
3a7ab618cba388a89f5187b2ad513667
0dd81dbe8d135accca359aa4ec360975
4a3168508f178e4d04b30d020cdabc59
f976684d52d2090b3530fd2e97a0964f
5f8f80c498238d6d02973c75abd4f9b5
Read 20 of 25...
8c0a3606161c1e32d8579b42964de802
e975e49d22046b1

In [3]:
ds=chancelator.sb.DataStore(trash=False)

idea count: 322
Database Schema Exists


In [4]:
ds.query_to_recordset("select tbl_name from sqlite_master")

[{'tbl_name': 'idea_fetch'},
 {'tbl_name': 'anno_fetch'},
 {'tbl_name': 'link_fetch'},
 {'tbl_name': 'fresh_ideas'},
 {'tbl_name': 'latest_user_content'}]

In [5]:
chancelator.save_ideas(ds,new_ideas)

Saving.
Saving.
Saving.
Saving.
Saving.
Saving.
Saving.
Saving.
Saving.
Saving.
Hashmatch - already cached
Hashmatch - already cached
Saving.
Saving.
Saving.
Hashmatch - already cached
Saving.
Hashmatch - already cached
Saving.
Saving.
Saving.
Hashmatch - already cached
Saving.
Saving.
Saving.


In [6]:
ds.query_to_recordset("select count(*) from idea_fetch")

[{'count(*)': 342}]

In [20]:

match_d = {"Hitler" : [re.compile(r"(?i)\b(hitler)\b"), 
                       re.compile(r"(?i)\b(führer|fuhrer)\b"),
                       re.compile(r"(?i)\b(Adolph)\b")
                      ],
           "Idi Amin" : [re.compile(r"(?i)\b(Idi Amin)\b")],
           "Genghis Khan" : [re.compile(r"(?i)\b(Genghis Khan)\b")],
           "Donald Trump" : [re.compile(r"(?i)\b(Donald Trump|Trump)\b")],
           "Brexit" : [re.compile(r"(?i)\b(Brexit|remoaner)\b")],
           "Political Correctness" : [re.compile(r"(?i)\b(social justice|sjw|cancel culture|political correctness)\b")]
           
          }


chancelator.perform_matches(ds, match_d)


It has been 9 days since Hitler was last mentioned.
anno found on Fishties by [zen_tom] //to Adolph Hitler caused me // on 17 Dec 2020

No mention of Idi Amin found in the cache.

It has been 73 days since Genghis Khan was last mentioned.
anno found on Hitler negative space by [xenzag] //posted the Genghis Khan-can as an // on 14 Oct 2020

It has been 2 days since Donald Trump was last mentioned.
anno found on Paper Shredder That Sounds Like It's Saying "YumYumYumYumYumYumYum" by [DenholmRicshaw] //If it eats Trump's tax returns// on 24 Dec 2020

It has been 1 days since Brexit was last mentioned.
idea_title found on BREXIT Whoopie Cushion by [Dub] //BREXIT Whoopie Cushion// on 25 Dec 2020

It has been 19 days since Political Correctness was last mentioned.
anno found on Big Brother's Mind Numbing Tea Biscuits by [zen_tom] //compliance and political correctness, parroting// on 07 Dec 2020


In [10]:
ds.query_to_recordset("select * from fresh_ideas")

[{'url': 'https://www.halfbakery.com/idea/_22Unedited_20documentary_22_20movie',
  'fetch_id': '638caaf3-ef0f-4c14-b2d4-c4bce01ed1c4',
  'hash': 'f14d10de4b55d4ce66a1e4ca424d3865',
  'fetch_date': 1608665612.236916},
 {'url': 'https://www.halfbakery.com/idea/Forget-Me-Do',
  'fetch_id': '355daa63-acac-4cff-b536-33cfa6a83a6f',
  'hash': 'de18d685646d4f8c3d0fb46cfb276e6e',
  'fetch_date': 1608665614.749482},
 {'url': 'https://www.halfbakery.com/idea/Boob_20friendly_20video_20compression_20layer',
  'fetch_id': '9d214ebb-ae2b-4c73-bfbe-a2ff841011d9',
  'hash': '938e1c97e22b857520dd2eb726f653c5',
  'fetch_date': 1608665615.347915},
 {'url': 'https://www.halfbakery.com/idea/Drugged_20headphones_20and_20earplugs',
  'fetch_id': 'fa6103fd-8b11-4c6c-b4f9-0e89ba2746cb',
  'hash': 'f390daf0a23fc211c05bf5e49586cc9f',
  'fetch_date': 1608665615.951941},
 {'url': 'https://www.halfbakery.com/idea/ii',
  'fetch_id': '2e25dee2-a3ca-402e-ba4e-57022d2cbd9c',
  'hash': 'cfca96001ef007b1489638a940ab7dda',

In [11]:
ds.query_to_recordset("select * from latest_user_content where fetch_id = '56867745-551b-45fb-96d2-1b5f42856086'")

[{'url': 'https://www.halfbakery.com/idea/BREXIT_20Whoopie_20Cushion',
  'fetch_id': '56867745-551b-45fb-96d2-1b5f42856086',
  'date': 1608854400.0,
  'user': 'Dub',
  'ctype': 'idea',
  'seq': -1,
  'text': 'One to be placed on every (socially distanced) seat in the houses of Parliament and Lords.Each cushion with the Field of Blue and EU stars on one side, and the Union Flag on the other (turned depending on Remain or Leave voter.)'},
 {'url': 'https://www.halfbakery.com/idea/BREXIT_20Whoopie_20Cushion',
  'fetch_id': '56867745-551b-45fb-96d2-1b5f42856086',
  'date': 1608854400.0,
  'user': '8th of 7',
  'ctype': 'anno',
  'seq': 0,
  'text': ' Can there be a version tastefully trimmed with hand-woven det cord, and a pressure-release detonator ?'},
 {'url': 'https://www.halfbakery.com/idea/BREXIT_20Whoopie_20Cushion',
  'fetch_id': '56867745-551b-45fb-96d2-1b5f42856086',
  'date': 1608854400.0,
  'user': 'Dub',
  'ctype': 'anno',
  'seq': 1,
  'text': " No [8th of 7], that sounds lik

In [14]:
#c = ds.connection.cursor()
#c.execute("DROP VIEW latest_user_content")

AttributeError: 'sqlite3.Cursor' object has no attribute 'commit'

In [16]:
#ds.connection.commit()

In [18]:
#c.execute("""
#                    CREATE VIEW latest_user_content
#                    as#
#
#                    select f.url, i.fetch_id, i.idea_date date, user, "idea_title" ctype, -3 seq, title text
#                                    from
#                                    idea_fetch i
#                                    join fresh_ideas f on i.fetch_id = f.fetch_id
#                    union all
#                        select f.url, i.fetch_id, i.idea_date date, user, "idea_description" ctype, -2 seq, description text
#                                    from
#                                    idea_fetch i
#                                    join fresh_ideas f on i.fetch_id = f.fetch_id
#                    union all
#                        select f.url, i.fetch_id, i.idea_date date, user, "idea" ctype, -1 seq, copy text
#                                    from
#                                    idea_fetch i
#                                    join fresh_ideas f on i.fetch_id = f.fetch_id
#
#                    union all
#                        select f.url, a.fetch_id, anno_date date, anno_user user, "anno" ctype, anno_seq seq, anno_text text
#                                    from
#                                    anno_fetch a
#                                    join fresh_ideas f on a.fetch_id = f.fetch_id
#                    union all
#                        select f.url, l.fetch_id, link_date date, link_user user, "link" ctype, link_seq seq, link_text || " " || link_anno text
#                                    from
#                                    link_fetch l
#                                    join fresh_ideas f on l.fetch_id = f.fetch_id
#                    order by date, seq, url
#            """)

In [21]:
ds.connection.commit()

In [22]:
ds.connection.close()